In [2]:
# Imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [5]:
train_dir = "data/train"
valid_dir = "data/val"

# Optional check
import os
assert os.path.exists(train_dir), f"Train directory not found: {train_dir}"
assert os.path.exists(valid_dir), f"Validation directory not found: {valid_dir}"

# Define ImageDataGenerators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)


Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [6]:
# CNN Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: fractured or not
])

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [8]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    verbose=1
)

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 169s 606ms/step - accuracy: 0.6221 - loss: 0.6339 - val_accuracy: 0.6817 - val_loss: 0.6181
Epoch 2/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 81s 292ms/step - accuracy: 0.7693 - loss: 0.4737 - val_accuracy: 0.7233 - val_loss: 0.6105
Epoch 3/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 85s 308ms/step - accuracy: 0.8768 - loss: 0.3069 - val_accuracy: 0.7350 - val_loss: 0.6363
Epoch 4/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 82s 296ms/step - accuracy: 0.9313 - loss: 0.1873 - val_accuracy: 0.7417 - val_loss: 0.7133
Epoch 5/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 80s 288ms/step - accuracy: 0.9595 - loss: 0.1251 - val_accuracy: 0.7367 - val_loss: 0.7203
Epoch 6/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 79s 287ms/step - accuracy: 0.9735 - loss: 0.0840 - val_accuracy: 0.7733 - val_loss: 0.7583
Epoch 7/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 81s 292ms/step - accuracy: 0.9780 - loss: 0.0676 - val_accuracy: 0.7783 - val_loss: 0.7363
Epoch 8/10
277/277 ━━━━━━━━━━━━━━━━━━━━ 89s 321ms/step - accuracy: 0.9855 - loss: 

In [ ]:
# Save the model
model.save("fracture_detection_model.h5")

In [ ]:
model = tf.keras.models.load_model("fracture_detection_model.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(tflite_model)

NameError: name 'tf' is not defined